<a href="https://colab.research.google.com/github/andrejadikic/Paralelni-algoritmi/blob/main/Dom3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=646530 sha256=bddbf1fda65ed42c2032b3422c3b931d9712af97a85c7db9aae1c3d1d7f5de24
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=4c1db3bd4b9a0c26c11c4928296ff1cdd8a5d83c019a48801b2de28454a24577
  Stored in directory: /root/.cache/pip/wheels/cb/fc/

In [21]:
import pycuda.driver as cuda
import pycuda.compiler as compiler
import pycuda.autoinit
import numpy as np
import math
from pycuda.compiler import SourceModule
# Define the kernel function in CUDA C


def popuni_patricu(m,n):
  matrix = [[0 for _ in range(m)] for _ in range(n)]

  for i in range(m):
    for j in range(n):
        matrix[i][j] = 0

  return matrix

mod = SourceModule("""
__global__ void smith_waterman(int *matrix, char *sequenceA, char *sequenceB, int rows, int cols,int trend) {
    // Calculate the row and column indices for the current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
  
    if(row==0 || col==0){
         return;
    }
    if(row+col!=trend){
      return;
    }
    // Check if the current thread is within the bounds of the matrix
    if (row >= rows || col >= cols) {
        return;
    }

    int scoreAbove = row > 0 ? matrix[(row - 1) * cols + col] : 0;
    int scoreLeft = col > 0 ? matrix[row * cols + (col - 1)] : 0;
    int scoreDiagonal = (row > 0 && col > 0) ? matrix[(row - 1) * cols + (col - 1)] : 0;
    int matchScore = sequenceA[row-1] == sequenceB[col-1] ? 5 : -3;



    matrix[row * cols + col] = max(0,max(scoreAbove - 9, max(scoreLeft - 9, scoreDiagonal + matchScore)));
}
""")
smith_waterman = mod.get_function("smith_waterman")

# Define the input arraysAAUGCCAUUGCCG
X="AAUGCCAUUGCCG"
Y="CAGCCUCGCUUAG"

matrix=np.array(popuni_patricu(len(X)+1, len(Y)+1))
X_bytes = X.encode()
Y_bytes = Y.encode()
# Allocate device memory for the input and output arrays
X_gpu = cuda.mem_alloc(len(X_bytes))
Y_gpu = cuda.mem_alloc(len(Y_bytes))
M_gpu = cuda.mem_alloc(matrix.nbytes)

# Transfer the input data to the device
cuda.memcpy_htod(X_gpu, X_bytes)
cuda.memcpy_htod(Y_gpu, Y_bytes)
cuda.memcpy_htod(M_gpu, matrix)
# Set the block and grid dimensions

block_dim = (1, 1, 1)
grid_dim = ( (len(X) // block_dim[0] + 1, len(Y) // block_dim[1] + 1))
#smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), block=block_dim, grid=grid_dim)


n = (len(X)+1)+(len(Y)+1)-1
for q in range(n):
  smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), np.int32(q), block = block_dim, grid = grid_dim)
# Allocate host memory for the output array
M = np.empty((len(X)+1, len(Y)+1), dtype=np.int32)

# Transfer the data back to the host
cuda.memcpy_dtoh(M, M_gpu)
print(M)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  0  0  0]
 [ 0  0  0  5  0  0  0  2  5  0  0  0  0  0]
 [ 0  5  0  0 10  5  0  5  0  0  0  0  0  0]
 [ 0  5  2  0  5 15  6  5  0  0  0  0  0  0]
 [ 0  0 10  1  0  6 12  0  0  0  0  0  0  0]
 [ 0  0  1  7  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [19]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule
import pycuda.compiler as compiler
import math

mod = SourceModule("""
__global__ void reduce_max(int *in, int *temp, int size)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int i = tid;
    temp[tid] = in[i];
    __syncthreads();

    for (int s = 1; s < blockDim.x; s *= 2)
    {
        if (tid % (2 * s) == 0)
        {
            temp[tid] = max(temp[tid], temp[tid + s]);
        }
        __syncthreads();
    }
}
""")
reduce_max = mod.get_function("reduce_max")


result = np.zeros(1).astype(np.int32)

matrix_gpu = drv.mem_alloc(M.nbytes)
result_gpu = drv.mem_alloc(result.nbytes)

drv.memcpy_htod(matrix_gpu, M)

block_size = 200

reduce_max(matrix_gpu, result_gpu, np.int32(matrix.size), block=(block_size, 1, 1), grid=(1, 1))
  
drv.memcpy_dtoh(result, result_gpu)

print(result[0])

18


In [22]:
import pycuda.driver as cuda
import pycuda.compiler as compiler
import pycuda.autoinit
import numpy as np
import math
from pycuda.compiler import SourceModule
from pycuda.driver import module_from_buffer
# Define the kernel function in CUDA C


def popuni_patricu(m,n):
  matrix = [[0 for _ in range(m)] for _ in range(n)]

  for i in range(m):
    for j in range(n):
        matrix[i][j] = 0

  return matrix

mod = SourceModule("""
  __constant__  char sequenceA[13];
  __constant__  char sequenceB[13];

__global__ void smith_waterman(int *matrix, int rows, int cols,int trend) {
    // Calculate the row and column indices for the current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
  
    if(row==0 || col==0){
         return;
    }
    if(row+col!=trend){
      return;
    }
    // Check if the current thread is within the bounds of the matrix
    if (row >= rows || col >= cols) {
        return;
    }

    int scoreAbove = row > 0 ? matrix[(row - 1) * cols + col] : 0;
    int scoreLeft = col > 0 ? matrix[row * cols + (col - 1)] : 0;
    int scoreDiagonal = (row > 0 && col > 0) ? matrix[(row - 1) * cols + (col - 1)] : 0;
    int matchScore = sequenceA[row-1] == sequenceB[col-1] ? 5 : -3;



    matrix[row * cols + col] = max(0,max(scoreAbove - 9, max(scoreLeft - 9, scoreDiagonal + matchScore)));
}
""")

# Compile the kernel function

smith_waterman = mod.get_function("smith_waterman")

# Define the input arrays
X="AAUGCCAUUGCCG"
Y="CAGCCUCGCUUAG"

matrix=np.array(popuni_patricu(len(X)+1, len(Y)+1))
X_bytes = X.encode()
Y_bytes = Y.encode()
# Allocate device memory for the input and output arrays

M_gpu = cuda.mem_alloc(matrix.nbytes)

# Transfer the input data to the device
A_gpu = mod.get_global("sequenceA")[0]
cuda.memcpy_htod(A_gpu,X_bytes)
B_gpu = mod.get_global("sequenceB")[0]
cuda.memcpy_htod(B_gpu, Y_bytes)

cuda.memcpy_htod(M_gpu, matrix)
# Set the block and grid dimensions

block_dim = (1, 1, 1)
grid_dim = ( (len(X) // block_dim[0] + 1, len(Y) // block_dim[1] + 1))
#smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), block=block_dim, grid=grid_dim)


n = (len(X)+1)+(len(Y)+1)-1
for q in range(n):
  smith_waterman(M_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), np.int32(q), block = block_dim, grid = grid_dim)
# Allocate host memory for the output array
M = np.empty((len(X)+1, len(Y)+1), dtype=np.int32)

# Transfer the data back to the host
cuda.memcpy_dtoh(M, M_gpu)

# Print the alignment matrix
print(M)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5]]


In [24]:
import pycuda.driver as cuda
import pycuda.compiler as compiler
import pycuda.autoinit
import numpy as np
import math
from pycuda.compiler import SourceModule
# Define the kernel function in CUDA C


def popuni_patricu(m,n):
  matrix = [[0 for _ in range(m)] for _ in range(n)]

  for i in range(m):
    for j in range(n):
        matrix[i][j] = 0

  return matrix

mod = SourceModule("""
__global__ void smith_waterman(int *matrix, char *sequenceA, char *sequenceB, int rows, int cols,int trend) {
    // Calculate the row and column indices for the current thread

    __shared__ char A_shared[13];
    __shared__ char B_shared[13];
  
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
  
    if(row==0 || col==0){
         return;
    }
    if(row+col!=trend){
      return;
    }
    // Check if the current thread is within the bounds of the matrix
    if (row >= rows || col >= cols) {
        return;
    }
    
    int ind = threadIdx.x;   
  
    A_shared[ind] = sequenceA[row-1];
    B_shared[ind] = sequenceB[col-1];

    int scoreAbove = row > 0 ? matrix[(row - 1) * cols + col] : 0;
    int scoreLeft = col > 0 ? matrix[row * cols + (col - 1)] : 0;
    int scoreDiagonal = (row > 0 && col > 0) ? matrix[(row - 1) * cols + (col - 1)] : 0;
    int matchScore = A_shared[ind] == B_shared[ind] ? 5 : -3;



    matrix[row * cols + col] = max(0,max(scoreAbove - 9, max(scoreLeft - 9, scoreDiagonal + matchScore)));
}
""")

# Compile the kernel function

smith_waterman = mod.get_function("smith_waterman")

# Define the input arraysAAUGCCAUUGCCG
X="AAUGCCAUUGCCG"
Y="CAGCCUCGCUUAG"

matrix=np.array(popuni_patricu(len(X)+1, len(Y)+1))
X_bytes = X.encode()
Y_bytes = Y.encode()
# Allocate device memory for the input and output arrays
X_gpu = cuda.mem_alloc(len(X_bytes))
Y_gpu = cuda.mem_alloc(len(Y_bytes))
M_gpu = cuda.mem_alloc(matrix.nbytes)

# Transfer the input data to the device
cuda.memcpy_htod(X_gpu, X_bytes)
cuda.memcpy_htod(Y_gpu, Y_bytes)
cuda.memcpy_htod(M_gpu, matrix)
# Set the block and grid dimensions

block_dim = (1, 1, 1)
grid_dim = ( (len(X) // block_dim[0] + 1, len(Y) // block_dim[1] + 1))
#smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), block=block_dim, grid=grid_dim)


n = (len(X)+1)+(len(Y)+1)-1
for q in range(n):
  smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), np.int32(q), block = block_dim, grid = grid_dim)
# Allocate host memory for the output array
M = np.empty((len(X)+1, len(Y)+1), dtype=np.int32)

# Transfer the data back to the host
cuda.memcpy_dtoh(M, M_gpu)

# Print the alignment matrix
print(M)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5]]


In [25]:
import pycuda.driver as cuda
import pycuda.compiler as compiler
import pycuda.autoinit
import numpy as np
import math
from pycuda.compiler import SourceModule
# Define the kernel function in CUDA C


def popuni_patricu(m,n):
  matrix = [[0 for _ in range(m)] for _ in range(n)]

  for i in range(m):
    for j in range(n):
        matrix[i][j] = 0

  return matrix

mod = SourceModule("""
__global__ void smith_waterman(int *matrix, char *sequenceA, char *sequenceB, int rows, int cols,int trend) {
    // Calculate the row and column indices for the current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
  
    if(row==0 || col==0){
         return;
    }
    if(row+col!=trend){
      return;
    }
    // Check if the current thread is within the bounds of the matrix
    if (row >= rows || col >= cols) {
        return;
    }

    int scoreAbove = row > 0 ? matrix[(row - 1) * cols + col] : 0;
    int scoreLeft = col > 0 ? matrix[row * cols + (col - 1)] : 0;
    int scoreDiagonal = (row > 0 && col > 0) ? matrix[(row - 1) * cols + (col - 1)] : 0;
    int matchScore = sequenceA[row-1] == sequenceB[col-1] ? 5 : -3;



    matrix[row * cols + col] = max(0,max(scoreAbove - 9, max(scoreLeft - 9, scoreDiagonal + matchScore)));
}
""")

# Compile the kernel function

smith_waterman = mod.get_function("smith_waterman")

# Define the input arraysAAUGCCAUUGCCG
X="AAUGCCAUUGCCGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
Y="CAGCCUCGCUUAGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

matrix=np.array(popuni_patricu(len(X)+1, len(Y)+1))
X_bytes = X.encode()
Y_bytes = Y.encode()
# Allocate device memory for the input and output arrays
X_gpu = cuda.mem_alloc(len(X_bytes))
Y_gpu = cuda.mem_alloc(len(Y_bytes))
M_gpu = cuda.mem_alloc(matrix.nbytes)

# Transfer the input data to the device
cuda.memcpy_htod(X_gpu, X_bytes)
cuda.memcpy_htod(Y_gpu, Y_bytes)
cuda.memcpy_htod(M_gpu, matrix)
# Set the block and grid dimensions

block_dim = (1024, 1, 1)
grid_dim = ( (len(X) // block_dim[0] + 1, len(Y) // block_dim[1] + 1))
#smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), block=block_dim, grid=grid_dim)


n = (len(X)+1)+(len(Y)+1)-1
for q in range(n):
  smith_waterman(M_gpu, X_gpu, Y_gpu, np.int32(len(X)+1), np.int32(len(Y)+1), np.int32(q), block = block_dim, grid = grid_dim)
# Allocate host memory for the output array
M = np.empty((len(X)+1, len(Y)+1), dtype=np.int32)

# Transfer the data back to the host
cuda.memcpy_dtoh(M, M_gpu)

# Print the alignment matrix
print(M)

/usr/local/lib/python3.8/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.8/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


[[    0     0     0 ...     0     0     0]
 [    0     0     5 ...     5     5     5]
 [    0     0     5 ...    10    10    10]
 ...
 [    0     0     5 ... 25530 25525 25525]
 [    0     0     5 ... 25533 25535 25530]
 [    0     0     5 ... 25539 25538 25540]]
